In [6]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError, pauli_error, depolarizing_error, thermal_relaxation_error)

## Quantum Errors

#### Construct a 5% single-qubit Bit-flip error

In [7]:
p_error = 0.05

bit_flip = pauli_error([ ('X', p_error), ('I', 1-p_error) ])
phase_flip = pauli_error([ ('Z', p_error), ('I', 1-p_error) ])

print(bit_flip)
print(phase_flip)

QuantumError on 1 qubits. Noise circuits:
  P(0) = 0.05, Circuit = 
   ┌───┐
q: ┤ X ├
   └───┘
  P(1) = 0.95, Circuit = 
   ┌───┐
q: ┤ I ├
   └───┘
QuantumError on 1 qubits. Noise circuits:
  P(0) = 0.05, Circuit = 
   ┌───┐
q: ┤ Z ├
   └───┘
  P(1) = 0.95, Circuit = 
   ┌───┐
q: ┤ I ├
   └───┘


#### Compose two bit-flip and phase-flip errors

In [9]:
bitphase_flip = bit_flip.compose(phase_flip)
print(bitphase_flip)

QuantumError on 1 qubits. Noise circuits:
  P(0) = 0.0025000000000000005, Circuit = 
   ┌───┐┌───┐
q: ┤ X ├┤ Z ├
   └───┘└───┘
  P(1) = 0.0475, Circuit = 
   ┌───┐┌───┐
q: ┤ X ├┤ I ├
   └───┘└───┘
  P(2) = 0.0475, Circuit = 
   ┌───┐┌───┐
q: ┤ I ├┤ Z ├
   └───┘└───┘
  P(3) = 0.9025, Circuit = 
   ┌───┐┌───┐
q: ┤ I ├┤ I ├
   └───┘└───┘


#### Tensor product errors

In [10]:
tensor_error = phase_flip.tensor(bit_flip)
print(tensor_error)

QuantumError on 2 qubits. Noise circuits:
  P(0) = 0.0025000000000000005, Circuit = 
     ┌───┐
q_1: ┤ Z ├
     ├───┤
q_0: ┤ X ├
     └───┘
  P(1) = 0.0475, Circuit = 
     ┌───┐
q_1: ┤ Z ├
     ├───┤
q_0: ┤ I ├
     └───┘
  P(2) = 0.0475, Circuit = 
     ┌───┐
q_1: ┤ I ├
     ├───┤
q_0: ┤ X ├
     └───┘
  P(3) = 0.9025, Circuit = 
     ┌───┐
q_1: ┤ I ├
     ├───┤
q_0: ┤ I ├
     └───┘


### Converting to and from QuantumChannel Operators

In [11]:
bit_flip_kraus = Kraus(bit_flip)
bit_flip_kraus

Kraus([[[ 9.74679434e-01+0.j,  0.00000000e+00+0.j],
        [-1.20234617e-16+0.j,  9.74679434e-01+0.j]],

       [[ 2.62045272e-16+0.j,  2.23606798e-01+0.j],
        [ 2.23606798e-01+0.j, -2.84112242e-16+0.j]]],
      input_dims=(2,), output_dims=(2,))

In [12]:
phase_flip_sop = SuperOp(phase_flip)
phase_flip_sop

SuperOp([[1. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
         [0. +0.j, 0.9+0.j, 0. +0.j, 0. +0.j],
         [0. +0.j, 0. +0.j, 0.9+0.j, 0. +0.j],
         [0. +0.j, 0. +0.j, 0. +0.j, 1. +0.j]],
        input_dims=(2,), output_dims=(2,))

In [13]:
print(QuantumError(bit_flip_kraus))

QuantumError(bit_flip_kraus) == bit_flip

QuantumError on 1 qubits. Noise circuits:
  P(0) = 1.0, Circuit = 
   ┌───────┐
q: ┤ kraus ├
   └───────┘


True